In [1]:
import requests
from tqdm.notebook import tqdm
p = requests.get('https://api.proxyscrape.com/v2/?request=displayproxies&protocol=http&timeout=10000&country=all&ssl=all&anonymity=all')
proxies = p.text
proxies = proxies.split('\n')
proxies = [p.replace('\r', '') for p in proxies]
proxies = [p for p in proxies if p != '']

from itertools import cycle
proxy_pool = cycle(proxies)
proxy = next(proxy_pool)

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import random

#load ids.txt
with open('ids.txt') as f:
    ids = f.readlines()
ids = [x.strip() for x in ids]
ids = [str(x) for x in ids]

print(len(list(set(ids))))

TESTING = False

#random sample of 30 ids
ids_sample = random.sample(ids, 30)

headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en,en-GB;q=0.9",
    "Cache-Control": "no-cache",
    "Connection": "keep-alive",
    "Cookie": "SSESSc128199592f79081a0f8de92b9e4cacf=Ede1Q4tCjfh4IHTiaDzBhpWJEns_oyvz1tBVZAirPHA",
    "Dnt": "1",
    "Host": "studenttheses.universiteitleiden.nl",
    "Pragma": "no-cache",
    "Sec-Ch-Ua": '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": '"Windows"',
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}




38370


In [3]:
def get_details(soup):
    try:
        title = soup.find("dd", {"class": "title"}).text.strip()
    except:
        title = None
    
    try:
        container = soup.find("dd", {"class": "first author"})
        author = container.find("span", {"class": "name"}).text.strip()
        year = container.find("span", {"class": "year"}).text.strip().replace("(", "").replace(")", "")
    except:
        author = None
        year = None

    try:
        container = soup.find("dl", {"title": "Supervisors"})
        supervisors = container.find("dd")
        supervisors = supervisors.findAll('a')
        supervisors = [i.text.strip() for i in supervisors]
    except:
        supervisors = None

    try:
        container = soup.find("dl", {"title": "Faculty"})
        faculty = container.find("dd").text.strip()
    except:
        faculty = None

    try:
        container = soup.find("dd", class_='genre')
        a = container.findAll('a')
        type = a[0].text.strip()
        program = a[1].text.strip()
    except:
        type = None
        program = None

    try:
        container = soup.find("dl", {"title": "Language"})
        language = container.find("dd").text.strip()
    except:
        language = None

    try:
        container = soup.find("dd", {"class": "abstract"})
        abstract = container.find("span")
        abstract = abstract.findAll('span')[1]
        abstract = abstract.text.strip()
        #remove Show less
        abstract = abstract.split("Show less")[0]
    except:
        abstract = None

    try:
        container = soup.find("dd", {"class": "topics"})
        keywords = container.findAll("div", class_='tag')

        keywords = [i.text.strip() for i in keywords]
    except:
        keywords = None

    data = {
        'title': title,
        'author': author,
        'supervisors': supervisors,
        'date': year,
        'faculty': faculty,
        'program': program,
        'type': type,
        'language': language,
        'abstract': abstract,
        'keywords': keywords
    }

    return data



In [4]:
url = "https://studenttheses.universiteitleiden.nl/search"


num_chunks = 100
chunk_size = len(ids) // num_chunks
ids_chunks = [ids[i:i+chunk_size] for i in range(0, len(ids), chunk_size)]

#check that all course IDs are in the chunks
assert sum([len(i) for i in ids_chunks]) == len(ids)

def process(url):
    import time
    response = None
    while response is None:
        try:
            time.sleep(1)
            proxy = next(proxy_pool)
            response = requests.get(url, proxies={'http': 'http://'+proxy}, timeout=10, headers=headers)
        except:
            #print the url that failed and add it to the list of failed urls
            print(url)
            with open('failed_urls.txt', 'a') as f:
                f.write(url+'\n')
            return None
            
    soup = BeautifulSoup(response.text, 'html.parser')
    details = get_details(soup)
    return details

TESTING = False
for i in tqdm(range(len(ids_chunks))):
    details = []
    #if the file already exists then skip iteration
    try:
        pd.read_csv(f'chunks/course_details_{i}.csv')
        continue
    except:
        pass
    for j in tqdm(range(10 if TESTING else len(ids_chunks[i]))):
        url = "https://studenttheses.universiteitleiden.nl" + ids_chunks[i][j]
        try:
            result = process(url)
        except:
            result = None
        print(result)
        if result is not None:
            details.append(result)


    chunk_details_df = pd.DataFrame(details)
    chunk_details_df.to_csv(f'chunks/course_details_{i}.csv', index=False)



  0%|          | 0/101 [00:00<?, ?it/s]

  0%|          | 0/383 [00:00<?, ?it/s]

{'title': 'The Restoration of a Golden Age', 'author': 'Kralt, Joost', 'supervisors': ['Wakefield, Bryce'], 'date': '2015', 'faculty': 'Faculty of Humanities', 'program': 'Asian Studies (60 EC) (MA)', 'type': 'Master thesis', 'language': 'en', 'abstract': None, 'keywords': ['Populism', 'Restorationism', 'Wilders', 'Ishihara', 'Nationalism', 'History', 'Rhetoric', 'Discourse Analysis']}
{'title': 'European Economic Integration and Countries in Transition: Lessons from Portugal', 'author': 'Chráska, Filip', 'supervisors': ['Rood, Jan'], 'date': '2015', 'faculty': 'Faculty of Humanities', 'program': 'International Relations (MA)', 'type': 'Master thesis', 'language': 'en', 'abstract': 'Portugal was the least developed country to join the European Communities before the Eastern enlargement. Its economy was characterized by weak commercial ties to Europe, a large agricultural sector employing more than one fourth of the labor force, lacking infrastructure and capital-intensive industries, a

  0%|          | 0/383 [00:00<?, ?it/s]

{'title': 'Towards a Better Ghana: Unveiling Talk Radio in the Media landscape of Ghana', 'author': 'Wilke, Ileen', 'supervisors': ['Gewald, J.B.', 'Bruijn, M.'], 'date': '2015', 'faculty': 'Faculty of Humanities', 'program': 'African Studies (MA)', 'type': 'Master thesis', 'language': 'en', 'abstract': None, 'keywords': ['Radio', 'Africa', 'Ghana', 'Talk radio', 'Framing']}
{'title': 'Closet Children:\xa0Growing\xa0Up\xa0in\xa0Confinement in\xa0Twenty-\xadFirst\xa0Century\xa0Memoirs\xa0and\xa0Fiction', 'author': 'Chi, Xiao-Fang', 'supervisors': ['Newton, Michael'], 'date': '2015', 'faculty': 'Faculty of Humanities', 'program': 'Literary Studies (MA)', 'type': 'Master thesis', 'language': 'en', 'abstract': None, 'keywords': None}
{'title': "The Portrayal of Widowhood in John Webster's The Duchess of Malfi and Elizabeth Cary's The Tragedy of Mariam", 'author': 'Stuivenberg, Jenny', 'supervisors': ['Akkerman, N.N.W.'], 'date': '2015', 'faculty': 'Faculty of Humanities', 'program': 'Engel

...

{'title': 'The Effect of Intensive Semantic Therapy on Picture Naming in Chronic Aphasic Patients', 'author': 'Nieuwenhuize, Rosemary', 'supervisors': ['Cremers, C.L.J.M.', 'Schiller, N.O.'], 'date': '2014', 'faculty': 'Faculty of Humanities', 'program': 'Linguistics (MA)', 'type': 'Master thesis', 'language': 'en', 'abstract': None, 'keywords': ['aphasia', 'chronic aphasia', 'Constraint-Induced Therapy', 'picture-naming', 'BNT', 'naming deficits']}
{'title': "'Ik werk dus ik ben': Over de schrijver en de roman in de nieuwe wereld.", 'author': 'Brockhus, Femke', 'supervisors': ['Ieven, Bram'], 'date': '2014', 'faculty': 'Faculty of Humanities', 'program': 'Nederlandse taal en cultuur (BA)', 'type': 'Bachelor thesis', 'language': 'nl', 'abstract': None, 'keywords': ['Postmodernisme', "Writer's block", 'Grunberg', 'Zwagerman']}
{'title': 'The Byzantine state and the Dynatoi, A struggle for supremacy 867 - 1071', 'author': 'Vrijaldenhoven, Joeri', 'supervisors': ['Hoppenbrouwers, P.C.M.']

  0%|          | 0/383 [00:00<?, ?it/s]

{'title': "The Artist's Gray. An Analysis of The Picture of Dorian Gray and The Symbolism of its Art in The Victorian Age", 'author': 'Chander, R.', 'supervisors': ['Leeuwen, E.J. van'], 'date': '2014', 'faculty': 'Faculty of Humanities', 'program': 'Engelse taal en cultuur (BA)', 'type': 'Bachelor thesis', 'language': 'en', 'abstract': None, 'keywords': ['The Picture of Dorian Gray', 'Victorian Age', 'Oscar Wilde', 'Art', 'Social masks']}
{'title': 'Karbalāʾ: Het Makka voor de sjiieten', 'author': 'Bom, Joey', 'supervisors': ['Bruning, Jelle'], 'date': '2014', 'faculty': 'Faculty of Humanities', 'program': 'Midden-Oostenstudies (BA)', 'type': 'Bachelor thesis', 'language': 'nl', 'abstract': None, 'keywords': ['Karbala', 'Sjiisme', 'Pelgrimage', 'Ashura', 'Husayn', 'Islam', 'Irak', 'Najaf', 'Baath']}
{'title': 'Semantische eigenschappen van reduplicatie in Wolof', 'author': 'Jordanoska, Izabela', 'supervisors': ['Mous, Maarten'], 'date': '2014', 'faculty': 'Faculty of Humanities', 'pro

  0%|          | 0/70 [00:00<?, ?it/s]

{'title': 'Achter de troon, een vergelijking van machtige vrouwen in het Franse en Ottomaanse rijk in de zestiende en zeventiende eeuw', 'author': 'Weele, M.J.M. van', 'supervisors': ['Duindam, J.F.J.'], 'date': '2012', 'faculty': 'Faculty of Humanities', 'program': 'History (MA)', 'type': 'Master thesis', 'language': 'nl', 'abstract': None, 'keywords': ['French dynasty', 'Ottoman dynasty', 'Position of women']}
{'title': "'niet tegenstaande mijne meenigvuldige devoiren'. De ervaringen van Dirk van Domburg commissaris van de Hollandse handelsgemeenschap te Danzig, 1697-1712.", 'author': 'Karskens, Nicolien', 'supervisors': ['Ebben, M.A.'], 'date': '2012', 'faculty': 'Faculty of Humanities', 'program': 'History (MA)', 'type': 'Master thesis', 'language': 'nl', 'abstract': None, 'keywords': ['Dirck van Domburgh', 'Gdansk', 'diplomatieke geschiedenis', 'Republiek', 'Hollandse handelsgemeenschap', 'Oostzeehandel']}
{'title': 'Punk met een hoofdletter R. De politieke ideologie van Rotterdam

In [5]:
#load failed urls
with open('failed_urls.txt') as f:
    failed_urls = f.readlines()

failed_urls = [x.strip() for x in failed_urls]
failed_urls = [str(x) for x in failed_urls]

#scrape
failed_details = []
for url in tqdm(failed_urls):
    try:
        result = process(url)
    except:
        result = None
    if result is not None:
        failed_details.append(result)

failed_details_df = pd.DataFrame(failed_details)
failed_details_df.to_csv(f'failed_details.csv', index=False)


  0%|          | 0/614 [00:00<?, ?it/s]

https://studenttheses.universiteitleiden.nl/handle/1887/20228?solr_nav%5Bid%5D=46c0117f46f0468fcccf&solr_nav%5Bpage%5D=806&solr_nav%5Boffset%5D=18
https://studenttheses.universiteitleiden.nl/handle/1887/64353?solr_nav%5Bid%5D=cf0ffd3ab1cc72b0b37f&solr_nav%5Bpage%5D=426&solr_nav%5Boffset%5D=16
https://studenttheses.universiteitleiden.nl/handle/1887/65089?solr_nav%5Bid%5D=e58f3ee5afbef544e32e&solr_nav%5Bpage%5D=444&solr_nav%5Boffset%5D=3
https://studenttheses.universiteitleiden.nl/handle/1887/29954?solr_nav%5Bid%5D=0dc1f5f65147fd0d04c0&solr_nav%5Bpage%5D=731&solr_nav%5Boffset%5D=1


In [6]:
#the last 4 failed again, 
urls = [
    'https://studenttheses.universiteitleiden.nl/handle/1887/20228?solr_nav%5Bid%5D=46c0117f46f0468fcccf&solr_nav%5Bpage%5D=806&solr_nav%5Boffset%5D=18',
    'https://studenttheses.universiteitleiden.nl/handle/1887/64353?solr_nav%5Bid%5D=cf0ffd3ab1cc72b0b37f&solr_nav%5Bpage%5D=426&solr_nav%5Boffset%5D=16',
    'https://studenttheses.universiteitleiden.nl/handle/1887/65089?solr_nav%5Bid%5D=e58f3ee5afbef544e32e&solr_nav%5Bpage%5D=444&solr_nav%5Boffset%5D=3',
    'https://studenttheses.universiteitleiden.nl/handle/1887/29954?solr_nav%5Bid%5D=0dc1f5f65147fd0d04c0&solr_nav%5Bpage%5D=731&solr_nav%5Boffset%5D=1',

]

failed_details = []
for url in tqdm(urls):
    try:
        result = process(url)
    except:
        result = None
    if result is not None:
        failed_details.append(result)

failed_details_df = pd.DataFrame(failed_details)
failed_details_df.to_csv(f'failed_details2.csv', index=False)

  0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
#combine all the chunks
import glob
import pandas as pd
files = glob.glob('chunks/*.csv')
files = sorted(files)

df = pd.concat([pd.read_csv(f) for f in files])
df._append(pd.read_csv('failed_details.csv'))
df._append(pd.read_csv('failed_details2.csv'))
df.to_csv('theses.csv', index=False)